In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/kobo2020'
%ls


In [4]:
# -*- coding: utf-8 -*-
"""kobo_bert.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1pwU35uTpZq29_4kCFPfrTaC2mvo_eZnK
"""

!pip install transformers
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5


     |████████████████████████████████| 778kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 9.6MB/s 
     |████████████████████████████████| 890kB 29.1MB/s 
     |████████████████████████████████| 1.1MB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=17774730f5e968e79dcc03a89a4ebbabb38bd4ea4d29be803968b528cc04b4f4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
curl is already the newest version (7.58.0-2ubuntu3.9).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-44

In [5]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import transformers

#東北大学の
pretrained_model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# 事前学習済みモデルのトークナイザを使用
tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model_name)

# 形態素解析 (内部でMeCabを使用)
text1 = "今日はいい天気だね"
text2 = "明日は雨がふるかもしれませんね"

print("text1", tokenizer.tokenize(text1))
print("text2", tokenizer.tokenize(text2))

"""
text1 ['今日', 'は', 'いい', '天気', 'だ', 'ね']
text2 ['明日', 'は', '雨', 'が', 'ふる', 'かも', 'しれ', 'ませ', 'ん', 'ね']
"""

# BERTに入力する形式に変換
for_bert_inputs = tokenizer([text1, text2], padding=True, return_tensors="pt")#[text1,test2]でバッチ化、ptは
print("for_bert_inputs", for_bert_inputs)
"""
2はCSLトークン、0はパディング、3はセパレートSEP
for_bert_inputs {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3,     0,     0,
             0,     0],
        [    2, 11475,     9,  3741,    14,  8491,  4830,  6758,  6769,  1058,
          1852,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
"""
# input_ids: 単語をIDに変換した系列．padding済み
# token_type_ids： 2文からなるペアを入力した場合に，1文目と2文目を区別するための系列
# attention_mask： input_idsのpadding部分とそうでない部分を区別するための系列


# 参考．文のペアを入れる場合の例
text3 = "そうかな"
text4 = "違うと思います"
tmp = tokenizer([[text1, text2],[text3, text4]], padding=True, return_tensors="pt")
print("tmp", tmp)
"""
tmp {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3, 11475,     9,
          3741,    14,  8491,  4830,  6758,  6769,  1058,  1852,     3],
        [    2,  1778,    29,    18,     3,  5720,    13,  2502,  2610,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],#0が一文目、1が二文目という意味
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
"""


# BERTの事前学習済みモデルをロード．BertForSequenceClassificationは1文が与えられて分類を行うクラス．num_labelsでラベル数を指定
# 他にもいろいろ用意されている．使用例も書かれている→  https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
model = transformers.BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=5)

out = model(input_ids=for_bert_inputs["input_ids"], token_type_ids=for_bert_inputs["token_type_ids"], attention_mask=for_bert_inputs["attention_mask"])
print("out", out)

model = transformers.BertModel.from_pretrained(pretrained_model_name)
out = model(input_ids=for_bert_inputs["input_ids"], token_type_ids=for_bert_inputs["token_type_ids"], attention_mask=for_bert_inputs["attention_mask"])
# pooled_output = outputs[1]

# pooled_output = self.dropout(pooled_output)
# logits = self.classifier(pooled_output)
print("out", out[1].size())
"""
out (tensor([[-0.4694, -0.2888,  0.1584,  0.1443,  0.2474],
        [-0.4795, -0.2614,  0.0915,  0.1614,  0.1209]],
       grad_fn=<AddmmBackward>),)
"""

# あとは損失を計算していつもどおりbackpropすればOK



text1 ['今日', 'は', 'いい', '天気', 'だ', 'ね']
text2 ['明日', 'は', '雨', 'が', 'ふる', 'かも', 'しれ', 'ませ', 'ん', 'ね']
for_bert_inputs {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3,     0,     0,
             0,     0],
        [    2, 11475,     9,  3741,    14,  8491,  4830,  6758,  6769,  1058,
          1852,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tmp {'input_ids': tensor([[    2,  3246,     9,  2575, 11385,    75,  1852,     3, 11475,     9,
          3741,    14,  8491,  4830,  6758,  6769,  1058,  1852,     3],
        [    2,  1778,    29,    18,     3,  5720,    13,  2502,  2610,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1,

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

out (tensor([[-0.3033, -0.0025,  0.0603, -0.3786, -0.2937],
        [-0.3011,  0.0206,  0.1518, -0.2958, -0.3015]],
       grad_fn=<AddmmBackward>),)
out torch.Size([2, 768])


'\nout (tensor([[-0.4694, -0.2888,  0.1584,  0.1443,  0.2474],\n        [-0.4795, -0.2614,  0.0915,  0.1614,  0.1209]],\n       grad_fn=<AddmmBackward>),)\n'

In [ ]:
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence
#word2id_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/nlp100_chap9'#todo 80.pyで保存したファイルパスを指定
import numpy as np

#train_x はテキストのリスト、yはテンソルにしてもOK
#get_dataはテキストのリストとyはテンソルを返す
def get_data(fname):
    text_list = []
    ids_list = [] #label
    #i = 0
    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.split('\t')#todo lineからタイトルを取得
            text = line[0]
            text_list.append(text)
            ids = float(line[1])
            ids_list.append(ids)

    ids_list = torch.Tensor(ids_list)

    return  text_list,ids_list

class MyDataset(torch.utils.data.Dataset):
    def __init__(self,text_list, ids_list):
      self.length = len(ids_list)
      self.text_list = text_list
      self.ids_list = ids_list
      tmp = tokenizer(text_list, padding=True, return_tensors="pt")
      self.inputids = tmp["input_ids"]
      self.tokenids = tmp["token_type_ids"]
      self.attentionmask = tmp["attention_mask"]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):

      return self.inputids[idx],self.tokenids[idx],self.attentionmask[idx],self.ids_list[idx]#todo データセットだけ作成する、Inputids、tokenids、idsの三つを返せば良い


# データの準備
train_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/dev.tsv'#todo 学習用ファイルパス
text_list,ids_list = get_data(train_path)

dataset = MyDataset(text_list,ids_list)
print(dataset[0])
print(dataset[1])

In [33]:

emb_dim = 300
hidden_dim = 50
target_size = 4
batch_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = transformers.BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=5)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


def train(model, train_loader, len_train):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for data, token,attention_mask, target in train_loader:
        print(data)
        # GPUへ
        data = data.to(device)
        token = token.to(device)
        attention_mask = attention_mask.to(device)
        target = target.to(device)
        

        optimizer.zero_grad()
        pred = model(input_ids=data, token_type_ids=token, attention_mask=attention_mask)#todo modelで予測
        loss = loss_fn(pred,target)#todo lossを計算

        loss.backward()
        optimizer.step()

        total_loss += loss
        pred_labels = torch.max(pred,axis = 1).indices
        correct_num +=  (pred_labels == target).sum().item()#todo 予測の正解数をカウント

    batch_loss = total_loss/len_train #バッチごとの平均ロス
    acc = correct_num/len_train#todo 正解率
    return batch_loss, acc



len_train = len(ids_list)
for epoch in range(30):
    train_loss, train_acc = train(model, train_loader, len_train)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

tensor([[    2,  9346,   136,  ...,     0,     0,     0],
        [    2,    35,  4378,  ...,     0,     0,     0],
        [    2,   373,     6,  ...,     0,     0,     0],
        ...,
        [    2, 25035, 16312,  ...,     0,     0,     0],
        [    2,    35,   811,  ...,     0,     0,     0],
        [    2,    13,     9,  ...,     0,     0,     0]])


RuntimeError: ignored